In [1]:
import os
import sys

import numpy as np
import pandas as pd

import torch.nn.functional as F

In [22]:
def do_kaggle_metric(predict, truth, threshold=0.5):

    N = len(predict)
    predict = predict.reshape(N, -1)
    truth   = truth.reshape(N, -1)

    predict = predict > threshold
    truth   = truth > 0.5
    intersection = truth & predict
    union        = truth | predict
    iou = intersection.sum(1) / (union.sum(1) + 1e-8)

    #-------------------------------------------
    result = []
    precision = []
    is_empty_truth   = (truth.sum(1) == 0)
    is_empty_predict = (predict.sum(1) == 0)

    threshold = np.array([0.50, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95])
    for t in threshold:
        p = iou >= t

        tp  = (~is_empty_truth)  & (~is_empty_predict) & (iou > t)
        fp  = (~is_empty_truth)  & (~is_empty_predict) & (iou <= t)
        fn  = (~is_empty_truth)  & ( is_empty_predict)
        fp_empty = ( is_empty_truth)  & (~is_empty_predict)
        tn_empty = ( is_empty_truth)  & ( is_empty_predict)

        p = (tp + tn_empty) / (tp + tn_empty + fp + fp_empty + fn)

        result.append( np.column_stack((tp,fp,fn,tn_empty,fp_empty)) )
        precision.append(p)

    result = np.array(result).transpose(1, 2, 0)
    precision = np.column_stack(precision)
    precision = precision.mean(1)

    return precision, result, threshold


In [18]:
predicts = []
truths = []
for i in range(4):
    predicts.append(np.ones((256, 1600, 4)))
    truths.append(np.zeros((256, 1600, 4)))

In [19]:
predicts[0].shape

(256, 1600, 4)

In [20]:
predicts = np.concatenate(predicts).squeeze()
truths = np.concatenate(truths).squeeze()

In [21]:
predicts.shape

(1024, 1600, 4)

In [23]:
precision, _, _ = do_kaggle_metric(predicts, truths, 0.5)
precision = precision.mean()

In [24]:
precision

0.0

In [3]:
train = pd.read_csv('../input/train.csv')

In [4]:
train.head()

,ImageId_ClassId,EncodedPixels
0,0002cc93b.jpg_1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0002cc93b.jpg_2,NaN
2,0002cc93b.jpg_3,NaN
3,0002cc93b.jpg_4,NaN
4,00031f466.jpg_1,NaN


In [11]:
def make_split_label(x):
    if x['class_count'] == 0:
        return 0
    if x['class_count'] <= 2:
        return 5
    if str(x['1']) != 'nan':
        return 1
    if str(x['2']) != 'nan':
        return 2
    if str(x['3']) != 'nan':
        return 3
    if str(x['4']) != 'nan':
        return 4

steel_df = pd.read_csv(os.path.join('..', 'input', 'train.csv'))
steel_df['ImageId'], steel_df['ClassId'] = zip(*steel_df['ImageId_ClassId'].apply(lambda x: x.split('_')))
steel_df = pd.pivot_table(steel_df, index='ImageId', columns='ClassId', values='EncodedPixels', aggfunc=lambda x: x, dropna=False)
steel_df = steel_df.reset_index()
steel_df.columns = [str(i) for i in steel_df.columns.values]
steel_df['class_count'] = steel_df[['1', '2', '3', '4']].count(axis=1)
steel_df['split_label'] = steel_df[['1', '2', '3', '4', 'class_count']].apply(lambda x: make_split_label(x), axis=1)

In [12]:
steel_df.head()

,ImageId,1,2,3,4,class_count,split_label
0,0002cc93b.jpg,29102 12 29346 24 29602 24 29858 24 30114 24 3...,NaN,NaN,NaN,1,5
1,00031f466.jpg,NaN,NaN,NaN,NaN,0,0
2,000418bfc.jpg,NaN,NaN,NaN,NaN,0,0
3,000789191.jpg,NaN,NaN,NaN,NaN,0,0
4,0007a71bf.jpg,NaN,NaN,18661 28 18863 82 19091 110 19347 110 19603 11...,NaN,1,5


In [18]:
steel_df['class_count'].value_counts()

1    6239
0    5902
2     425
3       2
Name: class_count, dtype: int64